<left>
    <img src="http://epecora.com.br/DataFiles/PPGOLD_PPGMNE.png" width="800" alt="PPGOLD Data Science: INTRO"  />
</left>

# **Combinatorial Optimization - PPGMNE:**  

## Taks

* Program a random seacrh for the TSP problem

Steps:

1. Read the file (Code already given)
2. Program a function to compute the objetive function, having as input a solution and a distance matrix
3. Program a function to randomly generate a solution
4. Randomly generate `2*n` solutions and print the best


---

Marina's Results


In [1]:
# Libs in use

import os
import csv
import random
import itertools
import numpy as np
import pandas as pd
from multiprocessing import Pool

In [7]:
# reads if THAT csv file exists
def read_tsp_file(filename):
    if os.path.exists(filename):
        data = pd.read_csv(filename)
        return data
    else:
        print(f"File {filename} not found.")
        return None

#read all the files in a directory
def read_all_tsp_files(directory):
    tsp_files = {}
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            full_path = os.path.join(directory, filename)
            tsp_files[filename] = read_tsp_file(full_path)
    return tsp_files

#calculate the total distance from a specific solution using distance matrix
def calculate_total_distance(solution, distance_matrix):
    total_distance = 0
    n = len(solution)
    for i in range(n - 1):
        total_distance += distance_matrix[solution[i], solution[i + 1]]

    total_distance += distance_matrix[solution[-1], solution[0]]
    return total_distance

#generate a random solution for this problem
def generate_random_solution(n):
    solution = list(range(n))
    random.shuffle(solution)
    return solution
#perform random search to find the best solution based on a number of generated solutions
def random_search(tsp_distance, num_solutions):
    n = tsp_distance.shape[0]
    S_best = None
    OF_best = np.inf

    for _ in range(num_solutions):

        S_random = generate_random_solution(n)
        

        OF_random = calculate_total_distance(S_random, tsp_distance)
        

        if OF_random < OF_best:
            S_best = S_random
            OF_best = OF_random

    return S_best, OF_best
# my directory where i did save the files
directory = r"C:\Users\55489\Documents\GitHub\combinatorial_optimization\TSP"
#reads all
tsp_files = read_all_tsp_files(directory)
#Process each TSP file read
for filename, tsp_data in tsp_files.items():
    if tsp_data is not None:
        print(f"\nProcessing file: {filename}")
        tsp_data.drop(columns=['X','Y'], inplace=True)
        tsp_distance = tsp_data.to_numpy()


        n = tsp_distance.shape[0]
        

        best_solution, best_distance = random_search(tsp_distance, num_solutions=2*n)

        print("Best solution found:", best_solution)
        print("Total distance of the best solution:", best_distance)




Processing file: TSP_05_001.csv
Best solution found: [1, 2, 4, 0, 3]
Total distance of the best solution: 2.2624739970950403

Processing file: TSP_05_002.csv
Best solution found: [3, 2, 0, 4, 1]
Total distance of the best solution: 2.417446791208771

Processing file: TSP_05_003.csv
Best solution found: [0, 2, 1, 3, 4]
Total distance of the best solution: 2.5607633112759287

Processing file: TSP_05_004.csv
Best solution found: [1, 3, 2, 4, 0]
Total distance of the best solution: 2.7115124840905467

Processing file: TSP_05_005.csv
Best solution found: [2, 0, 4, 3, 1]
Total distance of the best solution: 3.28807593824426

Processing file: TSP_100_001.csv
Best solution found: [29, 27, 41, 2, 8, 85, 45, 69, 91, 46, 97, 93, 11, 44, 74, 6, 9, 94, 50, 17, 16, 98, 83, 7, 72, 26, 32, 1, 34, 95, 56, 15, 87, 62, 36, 20, 43, 14, 68, 96, 89, 18, 33, 75, 3, 12, 60, 73, 31, 10, 82, 88, 5, 25, 42, 64, 57, 24, 59, 65, 47, 49, 28, 55, 51, 71, 13, 78, 99, 0, 67, 37, 23, 92, 66, 81, 80, 84, 70, 58, 77, 19


### **Combinatorial Optimization - PPGMNE:**  

##### Taks

* Program a Local seacrh for the TSP problem

Define a ONE basic neighborhood for the TSP instances.
Implement Your Neighborhood Using Different Strategies: Implement, preferably in Python, your neighborhood using the ​first_improvement and best_improvement​ strategies.
Record Top Outcomes: Record your top outcomes in the provided table. Make sure to document the results of applying the neighborhood and optimization strategies.

Deliverables(DO NOT ZIP the files):
One file with your code, preferably, in Python and the description of your neighborhood commented in the code.
Fill up the solution table in the collaboration space.
One CVS file, named your_name_TSP_local.csv​ with four columns and as many rows as instances: instance : Name of the instance strategy : first_improvement or best_improvement​​ best_OF : The best objective function you have found best_SOL : best solution found, it must be in the format of a Python list like [1, 3, 5, 2, 0]​


---

Marina's Results

In [8]:
# reads TSP file and return the distance matrix
def read_tsp_file(filename):
    distance_matrix = []
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            reader = csv.reader(f)
            next(reader)  
            distance_matrix = [list(map(float, row[2:])) for row in reader]  
    else:
        print(f"File {filename} not found.")
        return None
    return distance_matrix

# reads all TSP files in a directory and return a dictionary of distance matrices
def read_all_tsp_files(directory):
    tsp_files = {}
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            full_path = os.path.join(directory, filename)
            tsp_files[filename] = read_tsp_file(full_path)
    return tsp_files

# calculate the total distance of a TSP tour
def calculate_total_distance(distance_matrix, tour):
    total_distance = 0
    for i in range(len(tour)):
        total_distance += distance_matrix[tour[i - 1]][tour[i]]
    return total_distance

# Generate the neighborhood by swapping 
def generate_neighborhood(tour):
    neighborhood = []
    for i in range(len(tour)):
        for j in range(i + 1, len(tour)):
            neighbor = tour[:]
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
            neighborhood.append(neighbor)
    return neighborhood

# First Improvement Strategy
def first_improvement(distance_matrix, initial_tour):
    current_tour = initial_tour[:]
    current_distance = calculate_total_distance(distance_matrix, current_tour)
    
    while True:
        neighborhood = generate_neighborhood(current_tour)
        improvement_found = False
        
        for neighbor in neighborhood:
            neighbor_distance = calculate_total_distance(distance_matrix, neighbor)
            if neighbor_distance < current_distance:
                current_tour = neighbor
                current_distance = neighbor_distance
                improvement_found = True
                break
        
        if not improvement_found:
            break
    
    return current_tour, current_distance

# Best Improvement Strategy
def best_improvement(distance_matrix, initial_tour):
    current_tour = initial_tour[:]
    current_distance = calculate_total_distance(distance_matrix, current_tour)
    
    while True:
        neighborhood = generate_neighborhood(current_tour)
        best_neighbor = None
        best_neighbor_distance = current_distance
        
        for neighbor in neighborhood:
            neighbor_distance = calculate_total_distance(distance_matrix, neighbor)
            if neighbor_distance < best_neighbor_distance:
                best_neighbor = neighbor
                best_neighbor_distance = neighbor_distance
        
        if best_neighbor_distance < current_distance:
            current_tour = best_neighbor
            current_distance = best_neighbor_distance
        else:
            break
    
    return current_tour, current_distance

# Solve TSP for each file in thefolder
def solve_tsp_for_instances(folder_path):
    results = []
    files = read_all_tsp_files(folder_path)  
    
    for filename, distance_matrix in files.items():
        if distance_matrix is not None:  
            instance_name = filename.split('.')[0]
            num_cities = len(distance_matrix)
            initial_tour = list(range(num_cities))
            random.shuffle(initial_tour)

            # first improvement application
            best_tour_fi, best_distance_fi = first_improvement(distance_matrix, initial_tour)
            results.append([instance_name, 'first_improvement', best_distance_fi, best_tour_fi])

            # best improvement application
            best_tour_bi, best_distance_bi = best_improvement(distance_matrix, initial_tour)
            results.append([instance_name, 'best_improvement', best_distance_bi, best_tour_bi])

    return results

# results to csv
def write_results_to_csv(results, output_file):
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['instance', 'strategy', 'best_OF', 'best_SOL'])
        writer.writerows(results)

# run
folder_path = r'C:\Users\55489\Documents\GitHub\combinatorial_optimization\TSP'  
results = solve_tsp_for_instances(folder_path)
write_results_to_csv(results, 'marina_ferreira_TSP_local.csv')